## Create feature vector for all audio tests

In [1]:
import glob, os, re
FEATURE_PATH = '/home/zad/hopkinspd_server/data/feature'
USER_PATTERN = 'roch|udall'
FEATURE_VERSION = '1'
# list all reaction data files
feature_index = {}
users = glob.glob(FEATURE_PATH + "/*")

for user in users:
    username = os.path.basename(user)
    if re.search(USER_PATTERN, username, re.IGNORECASE):
        user_path = os.path.join(FEATURE_PATH, user)
        tapping_feature_files = glob.glob(user_path + "/*/*audio*." + FEATURE_VERSION + ".csv")
        if len(tapping_feature_files) > 0:
            feature_index[username] = tapping_feature_files
            
total = 0
print 'user\t\tcount'
for user in feature_index:
    n = len(feature_index[user])
    print user, '\t', n
    total += n
print 'total:', total
                

user		count
roch0738 	278
roch0739 	146
roch0732 	135
roch0734 	3
roch0735 	289
roch0081 	78
roch0087 	121
roch0351 	16
roch0167 	89
roch0165 	2
roch0161 	1
roch0247 	1
roch0169 	100
roch0241 	1
roch0761 	107
roch1066 	8
roch1063 	32
roch0027 	1
roch1068 	1
roch0606 	2
roch0605 	8
roch810 	39
roch0011 	4
roch0017 	122
roch0409 	2
udall013 	83
roch0420 	16
udall017 	5
udall016 	201
roch0425 	2
roch0542 	182
roch0544 	7
roch1012 	66
roch1013 	1
roch1017 	71
roch1019 	1
roch0908 	1
roch0860 	22
roch0861 	2
roch0867 	20
roch0864 	34
roch0865 	235
roch0900 	2
roch0868 	1
roch0869 	4
roch0904 	3
roch0906 	2
roch0336 	49
roch0337 	10
roch0334 	4
roch0115 	129
roch0332 	3
roch0333 	7
roch0330 	329
roch1246 	9
roch0890 	1
roch0338 	64
roch0339 	1
roch1124 	24
roch1120 	4
roch1449 	2
roch0209 	46
roch1443 	5
roch0201 	2
roch0206 	125
roch0207 	148
roch0204 	3
roch0533 	146
roch1237 	14
roch1003 	15
roch0649 	1
roch0643 	13
roch0646 	202
roch0645 	4
roch0888 	1
Udall077 	28
roch0762 	2
roch0765 	

## Create audio feature data frame

In [2]:
import pandas as pd
# aggregate feature file into a feature data frame
frames = []
for user in feature_index:
    print 'user:', user
    if re.search(USER_PATTERN, username, re.IGNORECASE):
        for feature_file in feature_index[user]:
            feature = pd.read_csv(feature_file)
            feature['id'] = user.lower()
            test_info = os.path.basename(feature_file[:-6]).split('_')
            datetime = test_info[-2:]
            feature['test'] = test_info[1]
            feature['date'] = datetime[0]
            feature['time'] = datetime[1]
            feature['session'] = feature_file.split('/')[-2]
            frames.append(feature)
df = pd.concat(frames, ignore_index=True)
print df.head()


user: roch0738
user: roch0739
user: roch0732
user: roch0734
user: roch0735
user: roch0081
user: roch0087
user: roch0351
user: roch0167
user: roch0165
user: roch0161
user: roch0247
user: roch0169
user: roch0241
user: roch0761
user: roch1066
user: roch1063
user: roch0027
user: roch1068
user: roch0606
user: roch0605
user: roch810
user: roch0011
user: roch0017
user: roch0409
user: udall013
user: roch0420
user: udall017
user: udall016
user: roch0425
user: roch0542
user: roch0544
user: roch1012
user: roch1013
user: roch1017
user: roch1019
user: roch0908
user: roch0860
user: roch0861
user: roch0867
user: roch0864
user: roch0865
user: roch0900
user: roch0868
user: roch0869
user: roch0904
user: roch0906
user: roch0336
user: roch0337
user: roch0334
user: roch0115
user: roch0332
user: roch0333
user: roch0330
user: roch1246
user: roch0890
user: roch0338
user: roch0339
user: roch1124
user: roch1120
user: roch1449
user: roch0209
user: roch1443
user: roch0201
user: roch0206
user: roch0207
user: roch0

In [3]:
from sqlalchemy import create_engine
engine = create_engine('postgresql:///hopkinspd')
df.to_sql('audio_' + FEATURE_VERSION, engine, if_exists='replace', index=False)

In [4]:
%load_ext sql
%sql postgresql:///hopkinspd

u'Connected: None@hopkinspd'

In [5]:
%%sql
alter table audio_1 add column tsp timestamp;
update audio_1 set tsp = cast(date || ' ' || time as timestamp);
select * from audio_1 limit 100;

Done.
24630 rows affected.
100 rows affected.


length,amp_mean,amp_std,amp_p1_1,amp_p1_0,amp_p2_2,amp_p2_1,amp_p2_0,amp_dfa,pitch,pitch_std,pitch_p1_1,pitch_p1_0,pitch_p2_2,pitch_p2_1,pitch_p2_0,pitch_dfa,id,test,date,time,session,tsp
15.5,0.723350524902,0.108732466355,-0.00881500244141,0.89524307251,0.00335950878154,-0.139835844921,2.10130672508,0.618489228454,241.124780316,345.136880725,24.6253747545,257.442494573,-3.42609583376,158.243112271,-972.525909749,0.809420396962,roch0738,test0,20150426,073436,HopkinsPD_test_roch0738_LGE-hammerhead-352584062352841_20150426_073709,2015-04-26 07:34:36
16.0,0.563898423139,0.18727982007,0.0334570641611,-0.00487166760015,0.000886667131516,0.00331038168955,0.230095122252,1.58538447216,219.595782074,295.726199833,48.4613529067,-342.606568111,0.344375100887,36.7525994766,-251.347166376,1.7593823788,roch0738,test0,20150621,064016,HopkinsPD_test_roch0738_LGE-hammerhead-352584062352841_20150621_064320,2015-06-21 06:40:16
8.0,0.815507676866,0.196664672091,-0.0602432250977,1.83964250353,-0.014780845477,0.442305521119,-2.33348286947,11.0278375242,228.207381371,1.4352665495,-0.2870533099,233.565709822,0.130478777227,-4.72333173563,270.404217926,3.00086880349,roch0738,test0,20150424,122544,HopkinsPD_test_roch0738_LGE-hammerhead-352584062352841_20150424_122838,2015-04-24 12:25:44
2.5,0.474090576172,0.198632638769,-0.0925170898437,0.797900390625,0.0167999267578,-0.210116577148,0.982699584961,0.0,94.7275922671,68.2053705461,25.4042179262,45.6414762742,50.5931458699,-328.747803163,602.166080844,0.0,roch0738,test0,20150424,122453,HopkinsPD_test_roch0738_LGE-hammerhead-352584062352841_20150424_122838,2015-04-24 12:24:53
12.0,0.943580040565,0.00581615660515,-0.000974550351992,0.965020148309,-3.72552252435e-05,0.000664679558722,0.947510192444,1.13881352256,658.787346221,120.586046077,14.3132350953,310.112207652,-4.61335676626,217.300932811,-1858.16547249,1.09973369357,roch0738,test0,20150424,121920,HopkinsPD_test_roch0738_LGE-hammerhead-352584062352841_20150424_122838,2015-04-24 12:19:20
10.0,0.577156760476,0.246795827542,-0.0559875488281,1.4169699929,0.0140959199492,-0.478865147304,4.44759278197,2.1068206203,223.901581722,262.532469452,46.9723598019,-363.644352133,10.6290019295,-271.897698084,1921.59106272,-0.252348040649,roch0738,test0,20150812,062057,HopkinsPD_test_roch0738_LGE-hammerhead-352584062352841_20150812_062325,2015-08-12 06:20:57
16.0,0.695476531982,0.170471865521,-0.0271426930147,1.33332981783,0.00420752560057,-0.224896396242,3.56752591173,1.96027657566,180.843585237,295.332948088,-28.5892432544,1044.0297736,-2.57428322823,92.4020684724,-322.914620593,0.943209761087,roch0738,test0,20150425,075840,HopkinsPD_test_roch0738_LGE-hammerhead-352584062352841_20150425_080142,2015-04-25 07:58:40
11.5,0.706255594889,0.217486712363,-0.0553918585077,1.62022126027,0.000732886803138,-0.0795771230113,1.81101612468,1.94362753633,198.066783831,168.827084314,14.0615974535,15.5671218061,0.206471911217,7.24802438335,69.318642693,4.17863151174,roch0738,test0,20150605,063004,HopkinsPD_test_roch0738_LGE-hammerhead-352584062352841_20150605_063254,2015-06-05 06:30:04
9.5,0.86569519043,0.0999025686561,-0.0290107495857,1.17030806108,-0.00451741074071,0.0658548759692,0.709532165527,2.87460557754,275.571177504,528.298202129,-59.7070884593,1230.59753954,-33.1252995686,635.924202482,-2148.18301646,1.02750157087,roch0738,test0,20150927,075512,HopkinsPD_test_roch0738_LGE-hammerhead-352584062352841_20150927_075737,2015-09-27 07:55:12
9.5,0.461923217773,0.0596639370809,-0.0126751524029,0.595012318004,0.00284530177261,-0.0724264896277,0.88523309881,1.55729194737,228.207381371,0.0,-3.3025622416e-15,228.207381371,-1.71219252258e-15,3.15521674309e-14,228.207381371,-2.25723530799,roch0738,test0,20150615,082902,HopkinsPD_test_roch0738_LGE-hammerhead-352584062352841_20150615_083200,2015-06-15 08:29:02


In [6]:
%%sql
alter table audio_1 add column day_order int;
update audio_1 set day_order = row_number from (
    select id, date, time, row_number() over w from audio_1 
    window w as (partition by id, date order by time) order by id, date, time  
) as s where audio_1.id = s.id and audio_1.date = s.date and audio_1.time = s.time



Done.
24630 rows affected.


[]